In [1]:
# Mrityunjay kumar #
#underfit : high bias
#optmimum : generalised output over test set too(optimum accuracy without overfitting nor underfitting)
#overfit : high variance, 
#          extremely fitting the training data but might not be similar for the test case(accuracy highest)

In [2]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer
tf.compat.v1.disable_eager_execution()

In [3]:
#load data
digits = load_digits()
x = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [13]:
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None,):
    with tf.compat.v1.name_scope(n_layer):
        Weights = tf.compat.v1.Variable(tf.compat.v1.random_normal([in_size,out_size]),name='W')
        biases = tf.compat.v1.Variable(tf.compat.v1.zeros([1,out_size])+0.1,name='b')
        logit = tf.compat.v1.matmul(inputs,Weights) + biases
        logit = tf.compat.v1.nn.dropout(logit,keep_prob)  #drops some observations and keeps certain amount
        if activation_function is None:
            output = logit
        else:
            output = activation_function(logit,)
        tf.compat.v1.summary.histogram(n_layer+"/Output",output)
        return output


#define placeholder for inputs to network
#with tf.name_scope('inputs'):
keep_prob = tf.compat.v1.placeholder(tf.float32)
xs = tf.compat.v1.placeholder(tf.compat.v1.float32,[None,64])  
ys = tf.compat.v1.placeholder(tf.compat.v1.float32,[None,10])  

#add hidden layer
layer1 = add_layer(xs,64,50,'l1',tf.compat.v1.nn.tanh)#input size has one feature(unit)&the next hidden layer consists of 10 units

#add output layer
predicted = add_layer(layer1,50,10,'l2',tf.compat.v1.nn.softmax)#layer one had 10 units, output ys has 1 unit and since non linear regression
                                  #therefore, no activation function (try and put some activation function and 
                                  #check the results)
        
#error between prediction and real data
loss = tf.compat.v1.reduce_mean(-tf.compat.v1.reduce_sum(ys*tf.compat.v1.log(predicted),reduction_indices=[1]))
tf.compat.v1.summary.scalar("loss",loss)
training = tf.compat.v1.train.GradientDescentOptimizer(0.1).minimize(loss)  #0.1 is the learning rate, minimise the loss

#important
init = tf.compat.v1.initialize_all_variables()
sess = tf.compat.v1.Session()

merged = tf.compat.v1.summary.merge_all ()

trainwriter = tf.compat.v1.summary.FileWriter("dropoutlogs/train",sess.graph)
testwriter =  tf.compat.v1.summary.FileWriter("dropoutlogs/test",sess.graph)


sess.run(init)


for step in range(1000):
    sess.run(training,feed_dict={xs:x_train,ys:y_train,keep_prob:0.5})
    if(step%50==0):
        trainresult = sess.run(merged,feed_dict={xs:x_train,ys:y_train,keep_prob:1})
        testresult = sess.run(merged,feed_dict={xs:x_test,ys:y_test,keep_prob:1})
        trainwriter.add_summary(trainresult,step)
        testwriter.add_summary(testresult,step)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_23' with dtype float and shape [?,10]
	 [[node Placeholder_23 (defined at <ipython-input-11-8a42dcbe54f9>:19) ]]

Original stack trace for 'Placeholder_23':
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "/anaconda3/lib/python3.6/asyncio/base_events.py", line 422, in

In [3]:
import numpy as np

In [6]:
a = np.zeros(10000)

In [7]:
b=a[np.random.randint(0,10,1000)]

In [8]:
b

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0